In [2]:
!pip install opencv-python scikit-learn

In [3]:
from google.colab import drive
drive.mount('/content/drive/MyDrive/doencas')


Mounted at /content/drive


In [9]:
# Importar bibliotecas
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Função para carregar imagens de múltiplas pastas
def load_images_from_folder(base_path, image_size=(224, 224)):
    images = []
    labels = []
    class_names = sorted(os.listdir(base_path))
    class_to_idx = {cls: i for i, cls in enumerate(class_names)}

    for cls in class_names:
        class_folder = os.path.join(base_path, cls)
        if not os.path.isdir(class_folder):
            continue
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            if os.path.isfile(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, image_size)
                    images.append(img)
                    labels.append(class_to_idx[cls])

    return np.array(images), to_categorical(labels, num_classes=len(class_names)), class_names

# Defina o caminho correto no seu Google Drive
base_path = '/content/drive/MyDrive/doencas'

# Carregar os dados
X, y, class_names = load_images_from_folder(base_path)
print(f'Classes: {class_names}')
print(f'Total de imagens: {len(X)}')

# Normalizar
X = X / 255.0

# Dividir em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Criar o modelo base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adicionar camadas finais
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Congelar camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Salvar o modelo localmente
model.save('/content/hand_skin_model.h5')

# Salvar o modelo no Google Drive
model.save('/content/drive/MyDrive/hand_skin_model.h5')

print("Modelo treinado e salvo com sucesso!")

Classes: ['dyshidrosis', 'eczema']
Total de imagens: 95
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.4176 - loss: 0.9683 - val_accuracy: 0.6316 - val_loss: 0.8290
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.4449 - loss: 0.9117 - val_accuracy: 0.6316 - val_loss: 0.8022
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.4319 - loss: 0.8695 - val_accuracy: 0.5263 - val_loss: 0.7723
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.4151 - loss: 0.8982 - val_accuracy: 0.5263 - val_loss: 0.7448
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4685 - loss: 0.8138 - val_accuracy: 0.5263 - val_loss: 0.7213
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.4373 - loss: 0.8161 - val_accuracy: 0.5263 - val_loss: 0.7009
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.5025 - loss: 0.7183 - val_accuracy: 0.5789 - val_loss: 0.6828
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5378 - 

Modelo treinado e salvo com sucesso!


In [10]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model

# Defina as classes na ordem correta
class_names = ['dyshidrosis', 'eczema']

# Carregar o modelo
model = load_model('/content/hand_skin_model.h5')

test_folder = '/content/drive/MyDrive/eczema/eczema_1677'
for img_file in os.listdir(test_folder):
    img_path = os.path.join(test_folder, img_file)
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, (224, 224))
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        pred = model.predict(img)
        pred_class = np.argmax(pred, axis=1)[0]
        prob = pred[0][pred_class] * 100  # confiança em %
        print(f'Imagem: {img_file} → Predição: {class_names[pred_class]} ({prob:.2f}%)')


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Imagem: imagemteste.jpg → Predição: eczema (89.52%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Imagem: t-chapped-fissured-feet-34.jpg → Predição: dyshidrosis (69.11%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Imagem: t-Dyshidrosis-59.jpg → Predição: dyshidrosis (76.55%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Imagem: 0_1.jpg → Predição: eczema (87.56%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Imagem: 0_0.jpg → Predição: eczema (83.00%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Imagem: 0_15.jpg → Predição: eczema (82.72%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Imagem: 0_19.jpg → Predição: dyshidrosis (73.87%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Imagem: 0_22.jpg → Predição: dyshidrosis (72.84%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Imagem: 0_14.jpg → Predição: eczema (54.98%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Imagem: 0_21.jpg → Predição: eczema (79.52%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Imagem: 0_11.jpg → Predição: eczema (78.85%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34